In [293]:
from socket import *
# import the "regular expressions" module
import re

In [294]:
def get_http_resource(url, file_name):
    """
    Get an HTTP resource from a server
           Parse the URL and call function to actually make the request.

    :param url: full URL of the resource to get
    :param file_name: name of file in which to store the retrieved resource

    (do not modify this function)
    """

    # Parse the URL into its component parts using a regular expression.
    url_match = re.search('http://([^/:]*)(:\d*)?(/.*)', url)
    url_match_groups = url_match.groups() if url_match else []
    #    print 'url_match_groups=',url_match_groups
    if len(url_match_groups) == 3:
        host_name = url_match_groups[0]
        host_port = int(url_match_groups[1][1:]) if url_match_groups[1] else 80
        host_resource = url_match_groups[2]
        print('host name = {0}, port = {1}, resource = {2}'.format(host_name, host_port, host_resource))
        status_string = do_http_exchange(host_name.encode(), host_port, host_resource.encode(), file_name)
        print('get_http_resource: URL="{0}", status="{1}"'.format(url, status_string))
    else:
        print('get_http_resource: URL parse failed, request not sent')

In [295]:
# Write Helper functions here

In [296]:
def do_http_exchange(host, port, resource, file_name):
    """
    Get an HTTP resource from a server

    :param bytes host: the ASCII domain name or IP address of the server machine (i.e., host) to connect to
    :param int port: port number to connect to on server host
    :param bytes resource: the ASCII path/name of resource to get. This is everything in the URL after the domain name,
           including the first /.
    :param file_name: string (str) containing name of file in which to store the retrieved resource
    :return: the status code
    :rtype: int
    """
    
    tcp_socket = socket(AF_INET, SOCK_STREAM)
    connectionTuple = (host, port)
    tcp_socket.connect(connectionTuple)
    request = b'GET ' + resource + b' HTTP/1.1\r\nHost: ' + host + b'\r\n\r\n'
    tcp_socket.send(request)
    response = b''
    #response += tcp_socket.recv(15)
    http_response = repr(response)
    con_length = ''
    header = b''
    con = 0
    while True:
        header += tcp_socket.recv(1)
        if b'\r\n\r\n' in header:
            headers = ""
            headers = header.decode()
            headers.split("\r\n")
            print(headers)
            con_length += headers[110:115]
            con = int(con_length)
            print(con)
            break
        if b'200' in header:
            if b'Content-Length' in header:
                message = ''
                while True:
                    data = tcp_socket.recv(int(con))
                    if len(data) < 1:
                        break
                    message = message + data
                    print(message)
                   
            if b'Transfer-Encoding' in header:
                msg = ''
                while True:
                    size = ''
                    while True:
                        if '\r\n' in size:
                            break
                        data = tcp_socket.recv(1)
                        size += data.decode()

                        if size == '\r\n':
                            tcp_socket.recv(1)
                            break
                        chunkSize = int(size,16) #broken 
                        chunkint = int(chunkSize)
                        print(chunkSize)
                        if chunkSize == 0:
                            break
                        else:
                            data = tcp_socket.recv(chunkint)
                            print(data.decode())
                            msg = msg + data.decode()
            
    mes = repr(message)
    f = open(file_name, "w")
    f.write(message)
    f.close()
    # Open a tcp socket
    # Connect the socket to the host on the given port
    # Create a request as a bytes object
    # Send the request to the host
    # Receive the response for the host
        ## Get the first line of the header first
        ## Extract the message code (e.g. 404, 200)
        ## If 200 proceeed to read the rest of the header lines
    # If the header contains the Content-Length, then
        ## Read the number of bytes given by the content length value
        ## save the bytes to a file given by file_name
    # Else if the header contains the Transfer-Encoding with value chunks
        ## Read each chunk in (remember the number that comes in with the length of the chunck is ASCII hexadecimal numbers)
        ## Combine the chunks
        ## Decode the chunks as ASCII
        ## Write the ASCII to a file given by file_name

    return 500  # Replace this "server error" with the actual status code

In [297]:
"""
Tests the client on a variety of resources
"""

# These resource request should result in "Content-Length" data transfer
get_http_resource('http://www.httpvshttps.com/check.png', 'check.png')

# this resource request should result in "chunked" data transfer
get_http_resource('http://www.httpvshttps.com/','index.html')

# If you find fun examples of chunked or Content-Length pages, please share them with us!

host name = www.httpvshttps.com, port = 80, resource = /check.png
HTTP/1.1 200 OK
Server: nginx
Date: Wed, 06 Oct 2021 16:55:15 GMT
Content-Type: image/png
Content-Length: 1719
Last-Modified: Mon, 30 May 2016 17:51:56 GMT
Connection: keep-alive
ETag: "574c7dbc-6b7"
x-instance: rocket-dallas
x-powered-by: anthum.com
Accept-Ranges: bytes


1719
get_http_resource: URL="http://www.httpvshttps.com/check.png", status="500"
host name = www.httpvshttps.com, port = 80, resource = /


ValueError: invalid literal for int() with base 16: ':'